# ReverseNumber Example - two users

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_mount_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

# Create two users

In [3]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [4]:
run_dispatch_worker_thread(cls[0])
run_mount_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Mount worker started in attached thread for user  0376adb05025f8c77acc10ce58102ec21be9a5e14a70a51261860ca64fa4dda602

~~~ Mount Receiver Task ~~~
Mounted ce535836-37d1-4ebd-9fc5-bc14d325e10f at flows:reverse_number_atomic:mounts:033d08f436a4ef1bdf271e85073e8d3127b328889a012f061545ca27fd79aecba0:ce535836-37d1-4ebd-9fc5-bc14d325e10f
Mounted 82edc4e7-0946-401d-9ed8-87d370f9cac7 at flows:reverse_number_atomic:mounts:033d08f436a4ef1bdf271e85073e8d3127b328889a012f061545ca27fd79aecba0:82edc4e7-0946-401d-9ed8-87d370f9cac7

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: ce535836-37d1-4ebd-9fc5-bc14d325e10f
message_paths: ['push_tasks:9179acda-cf5d-4c3b-8080-b87c7e17e126:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: ce535836-37d1-4ebd-9fc5-bc14d325e10f
message_paths: ['push_tasks:b6aae067-b337-4c4b-bd4d-aac682449003

In [5]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=False,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

# USER 1

In [6]:
run_dispatch_worker_thread(cls[1])
run_mount_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Mount worker started in attached thread for user  033d08f436a4ef1bdf271e85073e8d3127b328889a012f061545ca27fd79aecba0

~~~ Mount Initiator Task ~~~

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c
message_paths: ['push_tasks:b7ac19c4-0568-4b4d-86d6-be6de00a9200:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c
message_paths: ['push_tasks:9e437c27-9634-4e5b-8744-7b9e2202a506:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'second_reverse_flow', 'initial_message': FlowMessage(message_id='9c1d1571-2b8c-442f-8651-285bce38809e', created_at='2024-03-16 03:50:34.235967593', created_by='Proxy_reverse_number_sequential', message_type='FlowMessage', data={'id': 0

In [7]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [8]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

Mounted a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c at flows:reverse_number_sequential:mounts:local:a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   flow_class_name
   singleton
   init
   default_dispatch_point
   parallel_dispatch
   mounts/
     033d08f436a4ef1bdf271e85073e8d3127b328889a012f061545ca27fd79aecba0/
       ce535836-37d1-4ebd-9fc5-bc14d325e10f/
         config_overrides
         state
         init
       82edc4e7-0946-401d-9ed8-87d370f9cac7/
         init
         config_overrides

All flow instances:
 ce535836-37d1-4ebd-9fc5-bc14d325e10f
 82edc4e7-0946-401d-9ed8-87d370f9cac7


In [11]:
status(cls[1])

 reverse_number_sequential/
   init
   singleton
   default_dispatch_point
   parallel_dispatch
   flow_class_name
   mounts/
     local/
       a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c/
         config_overrides
         state
         init

All flow instances:
 a32d1f28-0d24-4cb5-80ba-6cc85c1d5d3c
